In [1]:
from DiffRed import DiffRed as dr

In [2]:
from tensorflow.keras.datasets.fashion_mnist import load_data

2023-06-12 00:22:50.629632: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 00:22:50.705649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 00:22:51.655374: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
(x_train,y_train),(x_test,y_test)=load_data()

In [4]:
A=x_train

In [5]:
A.shape

(60000, 28, 28)

In [6]:
A=A.reshape((60000,784))

In [7]:
A.shape

(60000, 784)

In [8]:
A=A[:2000,:] #sample

In [9]:
from DiffRed.utils import data_scaler

In [10]:
A,_=data_scaler(A)

In [11]:
import numpy.linalg as LA
import numpy as np

In [12]:
LA.norm(A[0,:]),np.mean(A[0,:])

(0.9999999999999984, -1.1328806373726086e-18)

In [13]:
A.shape

(2000, 784)

In [16]:
DiffRed=dr(k1=5,k2=15,opt_metric='m1')

In [17]:
embedding=DiffRed.fit_transform(A=A,max_iter=1000)

(784, 5)


In [18]:
embedding.shape

(2000, 20)

In [19]:
DiffRed.metric #value of the metric after monte carlo search   

4.878376242878346e-05

In [20]:
from DiffRed.utils import stress

In [21]:
stress(A,embedding)

0.09604734779946222

In [23]:
from DiffRed.utils import m1

In [21]:
m1(A,embedding)

1.573686867550883e-05

## Using Fit and Transform on seperate Datasets

In [28]:
from tensorflow.keras.datasets.mnist import load_data as mnist_load_data

In [30]:
(x_train,y_train),(x_test,y_test)=mnist_load_data()

In [31]:
B=x_train[:2000]

In [33]:
B=B.reshape((2000,784))

In [34]:
B,_=data_scaler(B)

In [35]:
DiffRed.fit(A)

In [36]:
embed_B=DiffRed.transform(B)

(784, 5)


In [37]:
m1(B,embed_B)

0.06196631849042977

In [38]:
stress(B,embed_B)

0.1684061136477237

# Using Knee of PCA-Curve to find the right $k1$ and $k2$

In [14]:
from DiffRed.utils import opt_dimensions

In [17]:
k1,k2=opt_dimensions(A=A, target_dimension=10, energy_threshold=0.7)
k1,k2

(7, 3)

In [18]:
DiffRed=dr(k1=k1,k2=k2,opt_metric='m1')

In [19]:
embedding=DiffRed.fit_transform(A=A,max_iter=1000)

In [21]:
stress(A,embedding)

0.1824962172033024

In [24]:
m1(A,embedding)

7.333110291840583e-05

In [25]:
DiffRed=dr(k1=k1,k2=k2,opt_metric='stress')

In [26]:
embedding=DiffRed.fit_transform(A=A,max_iter=1000)

In [27]:
stress(A,embedding)

0.17685600927255668

In [28]:
m1(A,embedding)

0.009581560152495672